In [ ]:
from __future__ import print_function

import os, sys
module_path = os.path.abspath(os.path.join('../..'))
sys.path.append(module_path)

import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
import pandas as pd
import time
import pickle

from pycrcnn.he.HE import TFHEnuFHE
from pycrcnn.he.tfhe_value import TFHEValue
from pycrcnn.he.alu import *

In [ ]:
def calcScaleZeroPoint(min_val, max_val,num_bits):
  # Calc Scale and zero point
  qmin = -2.**(num_bits-1)
  qmax = 2.**(num_bits-1) - 1.

  scale = (max_val - min_val) / (qmax - qmin)

  initial_zero_point = qmin - min_val / scale
  
  zero_point = 0
  if initial_zero_point < qmin:
      zero_point = qmin
  elif initial_zero_point > qmax:
      zero_point = qmax
  else:
      zero_point = initial_zero_point

  zero_point = int(zero_point)

  return scale, zero_point

def quantize_tensor(x, num_bits, min_val=None, max_val=None):
    
    if not min_val and not max_val: 
      min_val, max_val = x.min(), x.max()

    qmin = -2.**(num_bits-1)
    qmax = 2.**(num_bits-1) - 1.

    scale, zero_point = calcScaleZeroPoint(min_val, max_val, num_bits)
    q_x = zero_point + x / scale
    q_x = q_x.clip(qmin, qmax)
    q_x = q_x.astype(float).round().astype(int)
    
    return q_x
  
def quantize_tensor2(x, num_bits, min_val=None, max_val=None):
    
    if not min_val and not max_val: 
        min_val, max_val = x.min(), x.max()

    qmin = -2.**(num_bits-1)
    qmax = 2.**(num_bits-1) - 1.

    
    x = x - min_val          # Allineo tutto l'array in modo che parta da 0
    x /= (max_val - min_val) # Lo scalo tra 0 e 1    
    x *= (qmax - qmin)       # Lo scalo tra 0 e 16
    x -= qmax                # Lo sfaso tra -8 e 7
    
    q_x = x.astype(float).round().astype(int)
    
    return q_x

In [ ]:
# Prepare Penguins dataset
penguins = pd.read_csv('../penguins_size.csv')
penguins = penguins.sample(frac=1, random_state=2)
penguins = penguins.dropna()

x_train, y_train = penguins.loc[:, ["island", "culmen_length_mm", "flipper_length_mm", "body_mass_g"]].values, penguins.iloc[:, :1].values
for i in range(len(y_train)):
  if y_train[i][0] == "Adelie":
    y_train[i][0] = 0
  elif y_train[i][0] == "Gentoo":
    y_train[i][0] = 1
  else:
    y_train[i][0] = 2

island, sex = {}, {}
countI, countS = 0, 0

for i in range(len(x_train)):
  # Island
  if x_train[i][0] in island:
    x_train[i][0] = island[x_train[i][0]]
  else:
    island[x_train[i][0]] = countI
    x_train[i][0] = countI
    countI += 1

x_train[:, 1] = quantize_tensor2(x_train[:, 1], 4)
x_train[:, 2] = quantize_tensor2(x_train[:, 2], 4)
x_train[:, 3] = quantize_tensor2(x_train[:, 3], 4)

train, val = 150, 64
x_val, y_val = x_train[train:train+val], y_train[train:train+val]
x_test, y_test = x_train[train+val:], y_train[train+val:]
y_train = np_utils.to_categorical(y_train).astype(int)*8
x_train, y_train = x_train[:train], y_train[:train]

In [ ]:
HE_Client = TFHEnuFHE(16)

with open("secret_key", "rb") as f:
    HE_Client.secret_key = HE_Client.ctx.load_secret_key(f)
    
with open("cloud_key", "rb") as f:
    HE_Client.cloud_key = HE_Client.ctx.load_cloud_key(f)

cloud_key = HE_Client.cloud_key
HE_Client.generate_vm(cloud_key)

In [ ]:
num1 = HE_Client.encrypt(1)
num2 = HE_Client.encode(6)
sum = num1+num2
mul = num1*num2

## Net Architecture

In [ ]:
SHRT_MAX = 32767
SHRT_MIN = (-SHRT_MAX - 1 )

def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [ ]:
def pocketTanh(matIn, bits, inDims, outDims):
    yMax = 128
    yMin = -127
    joints = [128, 75, 32, -31, -74, -127]
    divisor = (1 << bits) * inDims
    slopesInv = [yMax, 8, 2, 1, 2, 8, yMax]

    matOut = np.full((matIn.shape[0], outDims), yMax)
    matActvGradInv = np.full((matIn.shape[0], outDims), slopesInv[0])

    for i in range(len(matIn)):
      for j in range(len(matIn[i].squeeze())):
        x = matIn[i].squeeze()[j] // divisor
        if x < joints[0]:
          matOut[i][j] = x // 4
          matActvGradInv[i][j] = slopesInv[1]
        if x < joints[1]:
          matOut[i][j] = x
          matActvGradInv[i][j] = slopesInv[2]
        if x < joints[2]:
          matOut[i][j] = x * 2
          matActvGradInv[i][j] = slopesInv[3]
        if x < joints[3]:
          matOut[i][j] = x
          matActvGradInv[i][j] = slopesInv[4]
        if x < joints[4]:
          matOut[i][j] = x // 4
          matActvGradInv[i][j] = slopesInv[5]
        if x < joints[5]:
          matOut[i][j] = yMin
          matActvGradInv[i][j] = slopesInv[6]
    return matOut.astype(int), matActvGradInv

In [ ]:
class FCLayer:
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size, outLayer = False, debug=False):
      self.input_size = input_size
      self.output_size = output_size
      self.outLayer = outLayer
      self.debug = debug
      self.weights = np.zeros((input_size, output_size)).astype(int)
      self.bias = np.zeros((1, output_size)).astype(int)
      self.mDfaWeight = np.zeros((1, 1))

    # returns output for a given input
    def forward(self, input_data, check_overflow=False, bits_tfhe = 0):
        max_value = 2.**(bits_tfhe-1) - 1.
        min_value = -2.**(bits_tfhe-1)
        self.input = input_data
        dot = np.matmul(self.input, self.weights) + self.bias
        # dot = np.clip(dot, SHRT_MIN + 1, SHRT_MAX)
        # print("DOT:")
        # print(dot)
        if check_overflow:
          overflow = (dot[dot>max_value]).size
          overflow += (dot[dot<min_value]).size
          if overflow > 0:
            print("FC output "  + repr(self.input_size))
            print(overflow)
        self.output, self.matActvGradInv = pocketTanh(dot, 8, self.input_size, self.output_size)
        # print("OUTPUT")
        # print(self.output)
        # print("ACT GRAD")
        # print(self.matActvGradInv)
        return self.output

    def backward(self, lastLayerDeltasMat, lrInv, check_overflow=False, bits_tfhe=0):      
      mDeltas = self.computeDeltas(lastLayerDeltasMat, lrInv)
      # print("MDELTAS")
      # print(mDeltas)

      batchSize = len(mDeltas) # 1 for one item
    
      mWeightUpdate = np.matmul(self.input.T, mDeltas)

      #mWeightUpdate = np.clip(mWeightUpdate, SHRT_MIN + 1, SHRT_MAX)

      mWeightUpdate = (mWeightUpdate // lrInv).astype(int)

      # print("MWEIGHTS UPDATE")
      # print(mWeightUpdate)

      self.weights -= mWeightUpdate

      max_value = 2.**(bits_tfhe-1) - 1.
      min_value = -2.**(bits_tfhe-1)

      if check_overflow:
          overflow = (self.weights[self.weights>max_value]).size
          overflow += (self.weights[self.weights<min_value]).size
          if overflow > 0:
            print("Weights"  + repr(self.input_size))
            print(overflow)

      ones = np.ones((batchSize, 1)).astype(int)

      mBiasUpdate = np.matmul(mDeltas.T, ones)

      mBiasUpdate = np.clip(mBiasUpdate, SHRT_MIN + 1, SHRT_MAX)

      mBiasUpdate = (mBiasUpdate.T // lrInv).astype(int)

      self.bias -= mBiasUpdate

      if check_overflow:
          overflow = (self.bias[self.bias>max_value]).size
          overflow += (self.bias[self.bias<min_value]).size
          if overflow > 0:
            print("Bias "  + repr(self.input_size))
            print(overflow)

      return lastLayerDeltasMat

    def setRandomDfaWeight(self, mInDim, mOutDim):
      range = isqrt((12 * SHRT_MAX) / (mInDim + mOutDim))
      self.mDfaWeight = np.random.randint(-range, range, (mInDim, mOutDim))
 
    def computeDeltas(self, lastLayerDeltasMat, lrInv):
      if self.outLayer:
        mDeltas = np.floor_divide(lastLayerDeltasMat, self.matActvGradInv)
      else:
        if self.mDfaWeight.shape[0] != lastLayerDeltasMat.shape[1] and  self.mDfaWeight.shape[1] != self.weights.shape[1]: # 0 rows, 1 cols
          print("Initialized DFA")
          self.setRandomDfaWeight(lastLayerDeltasMat.shape[1], self.weights.shape[1])
        dot = np.matmul(lastLayerDeltasMat, self.mDfaWeight)
        mDeltas = np.floor_divide(dot, self.matActvGradInv)
      return mDeltas

In [ ]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # test
    def test(self, x_test, y_test, check_overflow=False, bits_tfhe=0):
      # sample dimension first
      samples = len(x_test)
      corr = 0
      for j in range(samples):
          # forward propagation
          pred = self.predict(x_test[j], check_overflow, bits_tfhe)

          if pred.argmax() == y_test[j]:
            corr += 1
      return corr / samples * 100

    # predict output for given input
    def predict(self, input_data, check_overflow=False, bits_tfhe=0):
        output = np.expand_dims(input_data, axis=0)
        for layer in self.layers:
            output = layer.forward(output, check_overflow, bits_tfhe)

        return output

    # train the network
    def fit(self, x_train, y_train, epochs, miniBatchSize, lrInv, check_overflow=False, bits_tfhe=0):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            sumLoss = 0
            sumLossDelta = 0
            epochNumCorrect = 0
            numIter = int(samples/miniBatchSize)

            for j in range(numIter):
                batchNumCorrect = 0
                idxStart = j * miniBatchSize
                idxEnd = idxStart + miniBatchSize

                miniBatchImages = x_train[idxStart:idxEnd]
                miniBarchTargets = y_train[idxStart:idxEnd]

                # forward propagation
                output = miniBatchImages
                
                for layer in self.layers:
                  output = layer.forward(output, check_overflow=check_overflow, bits_tfhe=bits_tfhe)

                sumLoss += batchL2Loss(miniBarchTargets, output)
                lossDeltaMat, sumLossDelta = batchL2LossDelta(miniBarchTargets, output)

                for r in range(miniBatchSize):
                    if miniBarchTargets[r].argmax() == output[r].argmax():
                        batchNumCorrect += 1
                
                for layer in reversed(self.layers):
                    layer.backward(lossDeltaMat, lrInv, check_overflow=check_overflow, bits_tfhe=bits_tfhe)
                
                epochNumCorrect += batchNumCorrect;

            # print("Epoch: " + repr(i))
            # print("SumLoss: " + repr(sumLoss))
            # print("EpochNumCorrect: " + repr(epochNumCorrect))
            # print("Accuracy: " + repr(epochNumCorrect/samples * 100) + " %")
            # print("")

In [ ]:
## Enc Net Architecture

In [ ]:
def encrypted_pocketTanh(matIn, bits, inDims, outDims):
    yMax = HE_Client.encode(128)
    yMin = HE_Client.encode(-127)
    joints = HE_Client.encode_matrix([128, 75, 32, -31, -74, -127])
    divisor = (1 << bits) * inDims
    slopesInv = HE_Client.encode_matrix([128, 8, 2, 1, 2, 8, 128])

    matOut = np.full((matIn.shape[0], outDims), yMax)
    matActvGradInv = np.full((matIn.shape[0], outDims), slopesInv[0])

    for i in range(len(matIn)):
      for j in range(len(matIn[i].squeeze())):
        x = matIn[i].squeeze()[j] / divisor

        lt0 = x < joints[0]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt0, (x / 4).value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt0, slopesInv[1].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt1 = x < joints[1]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt1, x.value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt1, slopesInv[2].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt2 = x < joints[2]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt2, (x * 2).value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt2, slopesInv[3].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt3 = x < joints[3]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt3, x.value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt3, slopesInv[4].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt4 = x < joints[4]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt4, (x / 4).value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt4, slopesInv[5].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt5 = x < joints[5]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt5, yMin.value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt5, slopesInv[6].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

    return matOut, matActvGradInv

In [ ]:
def scalarL2LossDelta(y, yHat):
    return (yHat - y)

def batchL2LossDelta(yMat, yHatMat):
    # Assumption: 1 input -> 1 scalar sumLoss value
    # for 1 output of dimention T, lossDeltaMat = (1, T)
    lossDeltaMat = np.full((yMat.shape[0], yMat.shape[1]), HE_Client.encode(0))
    # Per each input item
    for i in range(len(yMat)):
      for j in range(len(yMat[i])):
        scalarLossDelta = scalarL2LossDelta(yMat[i][j], yHatMat[i].squeeze()[j])
        lossDeltaMat[i][j] = scalarLossDelta

    # return sum! (average is meaningless)
    return lossDeltaMat

In [ ]:
# inherit from base class Layer
class EncryptedFCLayer:
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size, outLayer = False):
      self.input_size = input_size
      self.output_size = output_size
      self.outLayer = outLayer
      self.weights = np.zeros((input_size, output_size)).astype(int)
      self.bias = HE_Client.encode_matrix(np.zeros((1, output_size)).astype(int))
      self.mDfaWeight = np.zeros((1, 1)).astype(int)

    # returns output for a given input
    def forward(self, input_data):
        self.input = input_data        
        dot = np.matmul(self.input, self.weights) + self.bias
        output, self.matActvGradInv = encrypted_pocketTanh(dot, 8, self.input_size, self.output_size)
        return output

    def backward(self, lastLayerDeltasMat, lrInv):      
      mDeltas = self.computeDeltas(lastLayerDeltasMat, lrInv)
      batchSize = len(mDeltas) # 1 for one item

      mWeightUpdate = np.matmul(self.input.T, mDeltas)
      mWeightUpdate = mWeightUpdate / lrInv
      mWeightUpdate = mWeightUpdate.reshape(self.input_size, self.output_size)

      print(HE_Client.decrypt_matrix(mWeightUpdate))

      if type(self.weights.squeeze()[0][0]) is not TFHEValue:
        self.weights = HE_Client.encode_matrix(self.weights)
        
      self.weights -= mWeightUpdate

      ones = np.ones((batchSize, 1)).astype(int)
      mBiasUpdate = np.matmul(mDeltas.T, ones)
      mBiasUpdate = mBiasUpdate.T / lrInv
      self.bias -= mBiasUpdate

    def setRandomDfaWeight(self, mInDim, mOutDim):
      range = isqrt((12 * SHRT_MAX) / (mInDim + mOutDim))
      self.mDfaWeight = np.random.randint(-range, range, (mInDim, mOutDim))
 
    def computeDeltas(self, lastLayerDeltasMat, lrInv):
      if self.outLayer:
        mDeltas = np.divide(lastLayerDeltasMat, self.matActvGradInv)
      else:
        if self.mDfaWeight.shape[0] != lastLayerDeltasMat.shape[1] and  self.mDfaWeight.shape[1] != self.weights.shape[1]: # 0 rows, 1 cols
          print("Initialized DFA")
          self.setRandomDfaWeight(lastLayerDeltasMat.shape[1], self.weights.shape[1])
        dot = np.matmul(lastLayerDeltasMat, self.mDfaWeight)   
        mDeltas = np.divide(dot, self.matActvGradInv)
      return mDeltas

In [ ]:
class EncryptedNetwork:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)
    
    # serialize the network
    def serialize(self):
        for l in self.layers:
          if hasattr(l, "weights"):
            l.weights = HE_Client.serialize_matrix(l.weights)
            l.bias = HE_Client.serialize_matrix(l.bias)
            l.matActvGradInv = None
            l.input = None
            if not l.outLayer:
              l.mDfaWeight = HE_Client.serialize_matrix(l.mDfaWeight)
    
    # deserialize the network
    def deserialize(self):
        for l in self.layers:
          if hasattr(l, "weights"):
            l.weights = HE_Client.deserialize_matrix(l.weights)
            l.bias = HE_Client.deserialize_matrix(l.bias)
            if not l.outLayer:
              l.mDfaWeight = HE_Client.deserialize_matrix(l.mDfaWeight)

    # test
    def test(self, x_test, y_test):
      # sample dimension first
      samples = len(x_test)
      corr = HE_Client.encode(0)
      enc_x = HE_Client.encrypt_matrix(x_test)
      enc_y = HE_Client.encrypt_matrix(y_test)

      for j in range(samples):
          # forward propagation
          pred = self.predict(enc_x[j])
          corr = TFHEValue(HE_Client.vm.gate_mux(pred == enc_y[j][0], (corr + 1).value, corr.value), corr.vm, corr.n_bits)

      return corr

    # predict output for given input
    def predict(self, input_data):
        output = np.expand_dims(input_data, axis=0)
        for layer in self.layers:
            output = layer.forward(output)

        return encrypted_argmax(output[0])

    # train the network
    def fit(self, x_train, y_train, epochs, miniBatchSize, lrInv):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            epochNumCorrect = 0
            numIter = int(samples/miniBatchSize)

            for j in range(numIter):
                batchNumCorrect = 0
                idxStart = j * miniBatchSize
                idxEnd = idxStart + miniBatchSize

                miniBatchImages = HE_Client.encrypt_matrix(x_train[idxStart:idxEnd])
                miniBarchTargets = HE_Client.encrypt_matrix(y_train[idxStart:idxEnd])

                # forward propagation
                output = miniBatchImages
                
                start_time = time.time()

                for layer in self.layers:
                  output = layer.forward(output)

                end_time = time.time()

                print("End forward batch: " + repr(j))
                print("Computation time: ")
                hours, rem = divmod(end_time-start_time, 3600)
                minutes, seconds = divmod(rem, 60)
                print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
                print("")

                # sumLoss += batchL2Loss(miniBarchTargets, output);
                lossDeltaMat = batchL2LossDelta(miniBarchTargets, output)

                # dec_out = HE_Client.decrypt_matrix(output)
                # dec_targets = HE_Client.decrypt_matrix(miniBarchTargets)

                # for r in range(miniBatchSize):
                #     if dec_targets[r].argmax() == dec_out[r].argmax():
                #         batchNumCorrect += 1
                
                start_time = time.time()

                # backward propagation
                for layer in reversed(self.layers):
                    layer.backward(lossDeltaMat, lrInv)

                end_time = time.time()

                print("End backward batch: " + repr(j))
                print("Computation time: ")
                hours, rem = divmod(end_time-start_time, 3600)
                minutes, seconds = divmod(rem, 60)
                print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
                print("")

                epochNumCorrect += batchNumCorrect;

            print("Epoch: " + repr(i))
            # print("EpochNumCorrect: " + repr(epochNumCorrect))
            # print("Accuracy: " + repr(epochNumCorrect/samples * 100) + " %")
            print("")

## Cross Validation

In [ ]:
with open("encnet1_corr.pkl", "rb") as f:
  readNet1 = pickle.load(f)
  readCorr1 = HE_Client.deserialize(pickle.load(f))
  readNet1.deserialize()
  print(HE_Client.decrypt(readCorr1))

In [ ]:
with open("encnet2_corr.pkl", "rb") as f:
  readNet2 = pickle.load(f)
  readCorr2 = HE_Client.deserialize(pickle.load(f))
  readNet2.deserialize()
  print(HE_Client.decrypt(readCorr2))

In [ ]:
with open("encnet3_corr.pkl", "rb") as f:
  readNet3 = pickle.load(f)
  readCorr3 = HE_Client.deserialize(pickle.load(f))
  readNet3.deserialize()
  print(HE_Client.decrypt(readCorr3))

In [ ]:
with open("encnet4_corr.pkl", "rb") as f:
  readNet4 = pickle.load(f)
  readCorr4 = HE_Client.deserialize(pickle.load(f))
  readNet4.deserialize()
  print(HE_Client.decrypt(readCorr4))

In [ ]:
argmax = encrypted_argmax([readCorr1, readCorr2, readCorr3, readCorr4])
print(HE_Client.decrypt(argmax))

res = [HE_Client.encode(0), HE_Client.encode(0), HE_Client.encode(0), HE_Client.encode(0)]

for i in range(len(res)):
  enc_i = HE_Client.encode(i)
  res[i] = TFHEValue(HE_Client.vm.gate_mux(enc_i == argmax, HE_Client.encode(1).value, res[i].value), res[i].vm, res[i].n_bits)

In [ ]:
weights1 = [readNet1.layers[0].weights, readNet2.layers[0].weights, readNet3.layers[0].weights, readNet4.layers[0].weights]
bias1 = [readNet1.layers[0].bias, readNet2.layers[0].bias, readNet3.layers[0].bias, readNet4.layers[0].bias]

weights2 = [readNet1.layers[1].weights, readNet2.layers[1].weights, readNet3.layers[1].weights, readNet4.layers[1].weights]
bias2 = [readNet1.layers[1].bias, readNet2.layers[1].bias, readNet3.layers[1].bias, readNet4.layers[1].bias]

In [ ]:
res_net = Network()
res_net.add(FCLayer(4, 2))
res_net.add(FCLayer(2, 3, outLayer=True))

res_weights1 = np.full(weights1[0].shape, HE_Client.encode(0))
res_bias1 = np.full(bias1[0].shape, HE_Client.encode(0))

res_weights2 = np.full(weights2[0].shape, HE_Client.encode(0))
res_bias2 = np.full(bias2[0].shape, HE_Client.encode(0))

for i in range(len(res)):
  res_weights1 = encrypted_mux_matrix(res[i] == HE_Client.encode(1), weights1[i], res_weights1)
  res_bias1 = encrypted_mux_matrix(res[i] == HE_Client.encode(1), bias1[i], res_bias1)
  res_weights2 = encrypted_mux_matrix(res[i] == HE_Client.encode(1), weights2[i], res_weights2)
  res_bias2 = encrypted_mux_matrix(res[i] == HE_Client.encode(1), bias2[i], res_bias2)

res_net.layers[0].weights = res_weights1
res_net.layers[0].bias = res_bias1
res_net.layers[1].weights = res_weights2
res_net.layers[1].bias = res_bias2

In [ ]:
print("Layer 0:")
print(HE_Client.decrypt_matrix(res_net.layers[0].weights))
print(HE_Client.decrypt_matrix(res_net.layers[0].bias))

print("Layer 1:")
print(HE_Client.decrypt_matrix(res_net.layers[1].weights))
print(HE_Client.decrypt_matrix(res_net.layers[1].bias))

In [ ]:
print("Layer 0:")
print(HE_Client.decrypt_matrix(readNet1.layers[0].weights))
print(HE_Client.decrypt_matrix(readNet1.layers[0].bias))

print("Layer 1:")
print(HE_Client.decrypt_matrix(readNet1.layers[1].weights))
print(HE_Client.decrypt_matrix(readNet1.layers[1].bias))

In [ ]:
plain_net = Network()
plain_net.add(FCLayer(4, 2))
plain_net.add(FCLayer(2, 3, outLayer=True))

plain_net.layers[0].weights = HE_Client.decrypt_matrix(res_net.layers[0].weights)
plain_net.layers[0].bias = HE_Client.decrypt_matrix(res_net.layers[0].bias)
plain_net.layers[1].weights = HE_Client.decrypt_matrix(res_net.layers[1].weights)
plain_net.layers[1].bias = HE_Client.decrypt_matrix(res_net.layers[1].bias)

acc = plain_net.test(x_test, y_test)
print("Testing accuracy best net " + repr(HE_Client.decrypt(argmax)) + ": " + repr(acc) + " %")

In [ ]:
norms = []
for net in [readNet1, readNet2, readNet3, readNet4]:
  plain_net = Network()
  plain_net.add(FCLayer(4, 2))
  plain_net.add(FCLayer(2, 3, outLayer=True))

  plain_net.layers[0].weights = HE_Client.decrypt_matrix(net.layers[0].weights)
  plain_net.layers[0].bias = HE_Client.decrypt_matrix(net.layers[0].bias)
  plain_net.layers[1].weights = HE_Client.decrypt_matrix(net.layers[1].weights)
  plain_net.layers[1].bias = HE_Client.decrypt_matrix(net.layers[1].bias)

  normNet = 0
  for l in plain_net.layers:
    normNet += np.linalg.norm(l.weights)
    normNet += np.linalg.norm(l.bias)
  
  norms.append(normNet)
  print("Norm net: " + str(normNet))

In [ ]:
plain_net = Network()
plain_net.add(FCLayer(4, 2))
plain_net.add(FCLayer(2, 3, outLayer=True))

plain_net.layers[0].weights = HE_Client.decrypt_matrix(res_net.layers[0].weights)
plain_net.layers[0].bias = HE_Client.decrypt_matrix(res_net.layers[0].bias)
plain_net.layers[1].weights = HE_Client.decrypt_matrix(res_net.layers[1].weights)
plain_net.layers[1].bias = HE_Client.decrypt_matrix(res_net.layers[1].bias)

normResNet = 0
for l in plain_net.layers:
  normResNet += np.linalg.norm(l.weights)
  normResNet += np.linalg.norm(l.bias)

print("Norm res net: " + str(normResNet))

In [ ]:
for i in range(len(norms)):
  print("||theta_" + str(i+1) + " - theta_beta||_2 = " + str(norms[i]-normResNet))